In [1]:
import os
import SimpleITK as sitk
import pandas as pd

In [2]:
# Define the directory containing the NIfTI files
directory = "/home/gkolokolnikov/PhD_project/nf_segmentation_interactive/NFInteractiveSegmentationBenchmarking/data/raw/imagesTr"

### Check Data Orientation

In [3]:
# Function to get spacing and orientation info from a NIfTI file
def analyze_nifti(file_path):
    image = sitk.ReadImage(file_path)
    spacing = image.GetSpacing()
    size = image.GetSize()
    direction = image.GetDirection()
    return spacing, size, direction

In [4]:
# Iterate over the files and collect spacing and orientation data
data = []
for file_name in os.listdir(directory):
    if file_name.endswith(".nii.gz"):
        file_path = os.path.join(directory, file_name)
        spacing, size, direction = analyze_nifti(file_path)
        data.append({
            "File Name": file_name,
            "Spacing": spacing,
            "Size": size,
            "Orientation": direction
        })

# Create a DataFrame for better visualization
df = pd.DataFrame(data)

In [ ]:
# Original orientation
single_file = os.path.join(directory, "4_2018_P_564_T2.nii.gz")
image = sitk.ReadImage(single_file)
print("Image shape in SimpleITK: ", image.GetSize())
image_np = sitk.GetArrayFromImage(image)
print("Image shape in numpy: ", image_np.shape)

In [ ]:
# Check RSA orientation
image_RSA = sitk.DICOMOrient(image, "RSA")
print("Image RSA shape in SimpleITK: ", image_RSA.GetSize())
image_np_RSA = sitk.GetArrayFromImage(image_RSA)
print("Image RSA shape in numpy: ", image_np_RSA.shape)

In [ ]:
# Check SRA orientation
image_SRA = sitk.DICOMOrient(image, "SRA")
print("Image SRA shape in SimpleITK: ", image_SRA.GetSize())
image_np_SRA = sitk.GetArrayFromImage(image_SRA)
print("Image SRA shape in numpy: ", image_np_SRA.shape)

In DINs we aim at unified orientation that leads to numpy shape of ~(31, 2919, 800) => Use RSA

### Processing stages

In [19]:
# Function to reorient an image to RAS
def reorient_to_rsa(image):
    # Reorient the image to RAS
    orient_filter = sitk.DICOMOrientImageFilter()
    orient_filter.SetDesiredCoordinateOrientation("RSA")
    ras_image = orient_filter.Execute(image)
    return ras_image

# Function to resample an image to the desired spacing
def resample_image(image, spacing):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()

    # Compute new size based on the desired spacing
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / spacing[i])))
        for i in range(3)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(spacing)
    resampler.SetSize(new_size)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    return resampler.Execute(image)

# Function to apply Z-score normalization
def z_score_normalization(image):
    array = sitk.GetArrayFromImage(image)
    mean = array.mean()
    std = array.std()
    normalized_array = (array - mean) / std
    return sitk.GetImageFromArray(normalized_array, isVector=image.GetNumberOfComponentsPerPixel() > 1)


In [ ]:
# Test the processing
desired_spacing = (1.7, 1.7, 7.8)
image = sitk.ReadImage(single_file)
print("Shape of numpy image before processing: ", sitk.GetArrayFromImage(image).shape)

image_ras = reorient_to_rsa(image)
print("Shape of numpy image after re-orientation to RSA: ", sitk.GetArrayFromImage(image_ras).shape)
print(image.GetSpacing())
print(image_ras.GetSpacing())
image_resampled = resample_image(image_ras, desired_spacing)
print("Shape of numpy image after resampling: ", sitk.GetArrayFromImage(image_resampled).shape)

print("Min and max values before normalization: ", sitk.GetArrayFromImage(image_resampled).min(), sitk.GetArrayFromImage(image_resampled).max())

image_normalized = z_score_normalization(image_resampled)
print("Min and max values after normalization: ", sitk.GetArrayFromImage(image_normalized).min(), sitk.GetArrayFromImage(image_normalized).max())